# Recommendation System for Amazon Clothing Products
---
## 3. Popularity Based Recommendation System

*Author*: Mariam Elsayed

*Contact*: mariamkelsayed@gmail.com

*Notebook*: 3 of 5

*Previous Notebook*: `reviews_loading_preprocessing.ipynb`

*Next Notebook*: `content_rec.ipynb`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Table of Contents

* [Introduction](#intro)

* [Loading the Data](#loading)

* [Popularity-Based Recommendation System](#rec)

    * [Using `average_rating`](#average_rating)

    * [Including a Threshold](#threshold)

    * [Using `rank`](#rank)

* [Conclusion](#conc)

## Introduction <a class="anchor" id="intro"></a>

The first recommendation system we will consider is popularity-based. It is a user-independant recommendation system, meaning that we have no information about the user on what we could make recommendations about. So, we suggest the most popular items.

## Loading the Data <a class="anchor" id="loading"></a>

This recommendation system will use the products data frame with the review summary created in the last notebook.

In [2]:
# loading products dataframe
products_df = pd.read_csv('Data/products_summary.csv')

In [3]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475227 entries, 0 to 475226
Data columns (total 63 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   category                                   475227 non-null  object 
 1   description                                475065 non-null  object 
 2   title                                      475227 non-null  object 
 3   brand                                      475227 non-null  object 
 4   rank                                       475227 non-null  int64  
 5   asin                                       475227 non-null  object 
 6   price                                      475227 non-null  float64
 7   maincat_Luggage & Travel Gear              475227 non-null  bool   
 8   maincat_Backpacks                          475227 non-null  bool   
 9   maincat_Novelty & More                     475227 non-null  bool   
 10  maincat_

## Popularity-Based Recommendation System<a class="anchor" id="rec"></a>

For this recommendation system, we are going to be sorting by highest rating for products with a high number of reviews, and we will also investigating using `rank`.

### Using `average_rating` <a class="anchor" id="average_rating"></a>

Let's start with sorting by the highest average rating.

In [4]:
# Sorting by top 10 products with the highest rating
products_df.sort_values(by=['average_rating'], ascending=False).head(10)

,category,description,title,brand,rank,asin,price,maincat_Luggage & Travel Gear,maincat_Backpacks,maincat_Novelty & More,...,subcat_Girls,subcat_Boys,"subcat_Shoe, Jewelry & Watch Accessories",subcat_Jewelry Accessories,subcat_Shoe Care & Accessories,subcat_Contemporary & Designer,subcat_Travel Accessories,"subcat_Surf, Skate & Street",average_rating,total_reviews
112363,"['Clothing, Shoes & Jewelry', 'Women', 'Shoes'...",Beautiful high heel made for any special occas...,Pleaser Women's STDUS802/C/M Platform Dress Sa...,Pleaser,4121098,B00B472OE8,94.18,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
138987,"['Clothing, Shoes & Jewelry', 'Women', 'Watche...",Jared Coach Women's Watch|Mini Boyfriend|14501...,Coach Womens 14501699 Mini Boyfriend Silver To...,Coach,871137,B00DQZ0TPC,13.99,False,False,False,...,False,False,False,False,False,False,False,False,5.0,3
316570,"['Clothing, Shoes & Jewelry', 'Men', 'Jewelry'...",Colorline Royal Bliss Filigree Septum Clicker ...,Colorline Royal Bliss Filigree Septum Clicker ...,Cosmic - Septums,9485227,B00ZE6L8PG,8.60,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
316564,"['Clothing, Shoes & Jewelry', 'Women', 'Access...",A vintage style lace garter broadcasts your di...,Steampunk Marlow Leather Top Hat with Garter H...,Unknown,2175498,B00ZE64E2A,177.00,False,False,False,...,False,False,False,False,False,False,False,False,5.0,2
316563,"['Clothing, Shoes & Jewelry', 'Women', 'Access...",Distinction is the magic word that makes our E...,Steampunk El Dorado Leather Top Hat,Unknown,380895,B00ZE6391M,167.00,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
138979,"['Clothing, Shoes & Jewelry', 'Women', 'Access...",Look cool and confident with these contemporar...,Prada 33PS 1AB0A7 Black 33PS Ornate Square Sun...,Prada,9078073,B00DQYX3V0,450.00,False,False,False,...,False,False,False,False,False,False,False,False,5.0,2
316562,"['Clothing, Shoes & Jewelry', 'Women', 'Access...","', 'Brand: MarcolinStyle: 2047Gender: womensFr...",Marcolin 2047 Womens/Ladies Designer Full-rim ...,Marcolin,995705,B00ZE5BTTC,14.95,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
316558,"['Clothing, Shoes & Jewelry', 'Women', 'Access...","', 'Brand: MarcolinStyle: 2030Gender: unisexFr...",Marcolin 2030 Mens/Womens Designer Full-rim Ti...,Marcolin,709801,B00ZE5B776,19.95,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
316557,"['Clothing, Shoes & Jewelry', 'Women', 'Jewelr...",This brilliant item is crafted from magnificen...,14k Yellow and White Gold Solid 1.2mm Round Wh...,The World Jewelry Center,3044098,B00ZE51PK0,263.00,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1
138984,"['Clothing, Shoes & Jewelry', 'Women', 'Watche...",Watch Sizing Guide\nPlease Note: COACH™ items ...,COACH Women's Classic Signature Bracelet Watch...,Coach,3754634,B00DQYZBS8,189.99,False,False,False,...,False,False,False,False,False,False,False,False,5.0,1


The top reviews only have one review with a 5 star rating. This is not the most informative representation of a popular product. 

### Including a Threshold <a class="anchor" id="threshold"></a>

To overcome the issue above, we are going to set a threshold on how many reviews are needed to be considered in the recommendation, then we will recommend the products with the highest reviews.

From the description of the `total_reviews` column, 75% of the products have at least 6 reviews, and 5794 reviews is the highest number of reviews a product has had. Lets pick the threshold to be 500.   

In [5]:
products_df['total_reviews'].describe()

count    475227.000000
mean         12.144804
std          62.776638
min           1.000000
25%           1.000000
50%           2.000000
75%           6.000000
max        5794.000000
Name: total_reviews, dtype: float64

In [6]:
# filtering out products with less than 500 reviews
threshold = 500
products_with_many_reviews = products_df[products_df['total_reviews'] >= threshold]

# sorting by highest average rating
products_with_many_reviews.sort_values(by=['average_rating'], ascending=False).head(10)

,category,description,title,brand,rank,asin,price,maincat_Luggage & Travel Gear,maincat_Backpacks,maincat_Novelty & More,...,subcat_Girls,subcat_Boys,"subcat_Shoe, Jewelry & Watch Accessories",subcat_Jewelry Accessories,subcat_Shoe Care & Accessories,subcat_Contemporary & Designer,subcat_Travel Accessories,"subcat_Surf, Skate & Street",average_rating,total_reviews
199707,"['Clothing, Shoes & Jewelry', 'Shoe, Jewelry &...","', 'GREAT. ANOTHER CRUSHED HEEL.', 'Weve all r...",OrthoStep Metal Shoe Horn 7.5 inch - Made in t...,OrthoStep,3443,B00K0PZ1Y0,10.95,False,False,False,...,False,False,True,False,True,False,False,False,4.9,506
164924,"['Clothing, Shoes & Jewelry', 'Men', 'Accessor...",VinicioBelt prides itself in providing timeles...,Men's Holeless Leather Ratchet Click Belt - Tr...,VINICIOBELT,2010,B00GKDFOJS,19.85,False,False,False,...,False,False,False,False,False,False,False,False,4.8,592
328254,"['Clothing, Shoes & Jewelry', 'Luggage & Trave...",The Only Cruise Tag with a Zip Top Seal Enclos...,Shacke Cruise Tags Zip Seal - Etag Holders & S...,Shacke,21601,B011DDHZJG,10.99,True,False,False,...,False,False,False,False,False,False,True,False,4.8,668
290646,"['Clothing, Shoes & Jewelry', 'Boys', 'Jewelry...",Pick your favorite Cross color from menu above...,Horseshoe Nail Cross Necklaces -(Solid Color) ...,Horseshoe Crosses,74357,B00V5RV0ME,29.61,False,False,False,...,False,True,False,False,False,False,False,False,4.8,673
291967,"['Clothing, Shoes & Jewelry', 'Men', 'Accessor...",VinicioBelt prides itself in providing timeles...,Men's Holeless Leather Ratchet Click Belt - Tr...,VINICIOBELT,3339,B00VCQ09KW,19.85,False,False,False,...,False,False,False,False,False,False,False,False,4.8,565
142751,"['Clothing, Shoes & Jewelry', 'Shoe, Jewelry &...",Tired Of Bending Over When Putting On Your Sho...,"24"" Inch Extra Long Handled Shoehorn",Shacke,31750,B00E3K9MGQ,12.99,False,False,False,...,False,False,True,False,True,False,False,False,4.8,699
205800,"['Clothing, Shoes & Jewelry', 'Luggage & Trave...","', 'Pack More Clothes in a Small Space For You...",Shacke Pak - 4 Set Packing Cubes - Travel Orga...,Shacke,118,B00KKXCJQU,24.99,True,False,False,...,False,False,False,False,False,False,True,False,4.8,2406
64918,"['Clothing, Shoes & Jewelry', 'Luggage & Trave...","With three large packing cubes included, this ...",eBags Large Packing Cubes for Travel - 3pc Set,Unknown,29099,B005LXPSHG,5.35,True,False,False,...,False,False,False,False,False,False,True,False,4.8,508
16894,"['Clothing, Shoes & Jewelry', 'Boys', 'Jewelry...",Pick your favorite Cross color from menu above...,Horseshoe Nail Cross Necklaces -(Solid Color) ...,Horseshoe Crosses,70751,B001DQ5YZ6,52.77,False,False,False,...,False,True,False,False,False,False,False,False,4.8,693
443500,"['Clothing, Shoes & Jewelry', 'Men', 'Accessor...",*** Perfect Gift! Get it at Today\'s Price! **...,Mt. Eston RFID Blocking Trifold Bifold Mens Wa...,MT. ESTON,4209,B01EDI9U2S,27.99,False,False,False,...,False,False,False,False,False,False,False,False,4.8,590


### Using `rank` <a class="anchor" id="rank"></a>

The `rank` column depicts the ranking position of the product in the Clothing, Shoes & Jewelry category determined by Amazon's A9 product search algorithm. This rank is updated once an hour and updated at the same time. Let's use the rank to make a recommendation. 

In [7]:
products_df.sort_values(by=['rank'], ascending=True).head(10)

,category,description,title,brand,rank,asin,price,maincat_Luggage & Travel Gear,maincat_Backpacks,maincat_Novelty & More,...,subcat_Girls,subcat_Boys,"subcat_Shoe, Jewelry & Watch Accessories",subcat_Jewelry Accessories,subcat_Shoe Care & Accessories,subcat_Contemporary & Designer,subcat_Travel Accessories,"subcat_Surf, Skate & Street",average_rating,total_reviews
32321,"['Clothing, Shoes & Jewelry', 'Women', 'Clothi...",A t-shirt bra that won't dig or poke. Warner's...,Warner's Women's This is Not a Bra Full-Covera...,Warner's,2,B00378LDBM,55.03,False,False,False,...,False,False,False,False,False,False,False,False,4.3,34
5850,"['Clothing, Shoes & Jewelry', 'Men', 'Accessor...",A classic. You'll reach for it every time the ...,Carhartt Men's Acrylic Watch Hat,Unknown,4,B000KGOHLM,12.99,False,False,False,...,False,False,False,False,False,False,False,False,4.7,2238
27158,"['Clothing, Shoes & Jewelry', 'Men', 'Accessor...",A classic. You'll reach for it every time the ...,Carhartt Men's Acrylic Watch Hat,Unknown,5,B002G9UEG8,12.99,False,False,False,...,False,False,False,False,False,False,False,False,4.6,2264
330023,"['Clothing, Shoes & Jewelry', 'Men', 'Clothing...",Gildan is one of the world's largest verticall...,Gildan Men's Crew T-Shirt Multipack,Unknown,6,B011QQE1HE,39.94,False,False,False,...,False,False,False,False,False,False,False,False,4.4,299
246815,"['Clothing, Shoes & Jewelry', 'Men', 'Clothing...",About Hanes HanesBrands is a socially responsi...,Hanes Men's Pullover EcoSmart Fleece Hooded Sw...,Unknown,6,B00OSTNWW2,43.58,False,False,False,...,False,False,False,False,False,False,False,False,4.1,7
45128,"['Clothing, Shoes & Jewelry', 'Women', 'Shoes'...",The iconic clog that started a comfort revolut...,Crocs Men's and Women's Classic Clog | Comfor...,Unknown,7,B004I6H7SK,90.92,False,False,False,...,False,False,False,False,False,False,False,False,4.3,54
1824,"['Clothing, Shoes & Jewelry', 'Men', 'Clothing...",About Hanes HanesBrands is a socially responsi...,Hanes Men's Pullover EcoSmart Fleece Hooded Sw...,Unknown,7,B000BRFH1E,43.58,False,False,False,...,False,False,False,False,False,False,False,False,4.4,1036
1822,"['Clothing, Shoes & Jewelry', 'Men', 'Clothing...",About Hanes HanesBrands is a socially responsi...,Hanes Men's Pullover EcoSmart Fleece Hooded Sw...,Unknown,7,B000BRETA4,43.58,False,False,False,...,False,False,False,False,False,False,False,False,4.4,2249
66639,"['Clothing, Shoes & Jewelry', 'Women', 'Shoes'...",The iconic clog that started a comfort revolut...,Crocs Men's and Women's Classic Clog | Comfor...,Unknown,8,B005PQQ1VQ,90.92,False,False,False,...,False,False,False,False,False,False,False,False,4.5,3208
76059,"['Clothing, Shoes & Jewelry', 'Men', 'Clothing...",About Hanes HanesBrands is a socially responsi...,Hanes Men's Pullover EcoSmart Fleece Hooded Sw...,Unknown,9,B006WCLP18,43.58,False,False,False,...,False,False,False,False,False,False,False,False,4.6,248


Picking the highest ranks yields different results compared to using the ratings with a threshold. Half of the highest ranks wouldn't meet the threshold set above however the reviews for these products seem mostly good. I believe that the duplicate ranks was caused because the webscraping took place over a longer period of time while the rankings were determined every hours. Since the rankings fluctuate based on Amazon's algorithms, these recommendations could be incorparated to a hybrid model later.

## Conclusion <a class="anchor" id="conc"></a>

A simple popularity-based recommendation system with a threshold was made by considering products with a high number of reviews then by sorting by the average review rating. 

*Next notebook*: `content_rec.ipynb`